### Recursively Rename Files After Validation

### **Test before use on real data**

In [1]:
import os
import pandas as pd

In [2]:
# Define the root directory containing class folders
recs_directory = "/mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/it_9/random/"

In [3]:
# Function to rename clips based on the annotation map
def rename_clips(directory, annotation_map):
    # List all .wav files in the directory
    wav_files = [f for f in os.listdir(directory) if f.lower().endswith('.wav')]

    for wav_file in wav_files:
        if wav_file in annotation_map:
            annotation = annotation_map[wav_file]

            # Skip if annotation is NaN or empty
            if pd.isna(annotation) or str(annotation).strip() == "":
                continue
                
            new_name = f"{os.path.splitext(wav_file)[0]}_{annotation}.WAV"
            old_path = os.path.join(directory, wav_file)
            new_path = os.path.join(directory, new_name)

            try:
                # Rename the file
                os.rename(old_path, new_path)
                #print(f"Renamed: {wav_file} → {new_name}")
            except Exception as e:
                print(f"X Error renaming {wav_file}: {e}")
                

# Loop through all class folders within the root directory
for class_folder in os.listdir(recs_directory):
    class_path = os.path.join(recs_directory, class_folder)
    
    # Ensure it's a directory before proceeding
    if not os.path.isdir(class_path):
        continue  

    # Check for subfolders (`top_scoring` and `for_training`)
    subfolders = [os.path.join(class_path, subfolder) 
                  for subfolder in os.listdir(class_path) 
                  if os.path.isdir(os.path.join(class_path, subfolder))]

    if subfolders:
        # If subfolders exist, process each one separately
        for subfolder in subfolders:
            # Find the `_Validation.txt` file within the subfolder
            txt_files = [f for f in os.listdir(subfolder) if f.lower().endswith('_validation.txt')]
            if len(txt_files) == 0:
                print(f"No annotation file found in {subfolder}, skipping...")
                continue
            
            txt_file = os.path.join(subfolder, txt_files[0])

            try: 
                # Try to read the file and create an annotation map
                df_annotations = pd.read_csv(txt_file, delimiter='\t')
                annotation_map = dict(zip(df_annotations['Begin File'], df_annotations['Annotation']))

                # Rename files within the subfolder
                rename_clips(subfolder, annotation_map)
            except Exception as e:
                print(f"Error processing {txt_file}: {e}")
                continue
    else:
        # If no subfolders, look for the _Validation.txt directly in the class folder
        txt_files = [f for f in os.listdir(class_path) if f.lower().endswith('_validation.txt')]
        if len(txt_files) == 0:
            print(f"No annotation file found in {class_folder}, skipping...")
            continue
        
        txt_file = os.path.join(class_path, txt_files[0])

        try: 
            # Try to read the annotations and create a mapping     
            df_annotations = pd.read_csv(txt_file, delimiter='\t')
            annotation_map = dict(zip(df_annotations['Begin File'], df_annotations['Annotation']))

            # Rename files directly in the class folder
            rename_clips(class_path, annotation_map)
        except Exception as e:
            print(f"Error processing {txt_file}: {e}")
            continue

print("File renaming complete.")


No annotation file found in Rusty-barred Owl_song, skipping...
File renaming complete.
